In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import pickle
import gensim
import pandas as pd
import numpy as np

from collections import OrderedDict
from torch import nn, optim
from torchtext.data import utils
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [3]:
from dataset import AbstractDataset
from util import save_model, load_model_and_opt, batch_predict
from cnn_util import CNNBase1, train_model

In [4]:
DEVICE = (f'cuda:0' if torch.cuda.is_available() else 'cpu')
HOME = '/home/hice1/khom9/CSE 8803 BMI Final Project'
EMBED_KEYS_PATH = f'{HOME}/wordvectors/abstracts200_normalized.wordvectors'

In [5]:
# Type of CNN (1, 2, or 3); refer to cnn_util.py for more info
VERSION = 1

In [6]:
print(f'Using device {DEVICE}')
if torch.cuda.is_available():
    print(torch.cuda.get_device_name())

Using device cuda:0
NVIDIA L40S


In [7]:
# Only run once


from nltk.stem import WordNetLemmatizer
import nltk
from gensim.models import Word2Vec

'''
df = pd.read_csv('CleanedAVdata.csv')
nltk.download('wordnet')
tk = utils.get_tokenizer('spacy')
lemma = WordNetLemmatizer()
abstracts = df['Abstract']
embed_dim = 200
tokens = pd.Series([[lemma.lemmatize(w) for w in tk(abst)] for abst in abstracts])
model = Word2Vec(sentences=tokens, vector_size=embed_dim, window=5, min_count=1, workers=12)

mu = np.mean(model.wv.vectors)
sigma = np.sqrt(np.var(model.wv.vectors))
model.wv.vectors = (1 + (np.clip(model.wv.vectors, mu-3*sigma, mu+3*sigma) - mu) / (3*sigma)) / 2
model.wv.vectors = np.clip(model.wv.vectors, 0, 1)

model.wv['\0'] = np.zeros(embed_dim)
model.wv.save(EMBED_KEYS_PATH)
'''



"\ndf = pd.read_csv('CleanedAVdata.csv')\nnltk.download('wordnet')\ntk = utils.get_tokenizer('spacy')\nlemma = WordNetLemmatizer()\nabstracts = df['Abstract']\nembed_dim = 200\ntokens = pd.Series([[lemma.lemmatize(w) for w in tk(abst)] for abst in abstracts])\nmodel = Word2Vec(sentences=tokens, vector_size=embed_dim, window=5, min_count=1, workers=12)\n\nmu = np.mean(model.wv.vectors)\nsigma = np.sqrt(np.var(model.wv.vectors))\nmodel.wv.vectors = (1 + (np.clip(model.wv.vectors, mu-3*sigma, mu+3*sigma) - mu) / (3*sigma)) / 2\nmodel.wv.vectors = np.clip(model.wv.vectors, 0, 1)\n\nmodel.wv['\x00'] = np.zeros(embed_dim)\nmodel.wv.save(EMBED_KEYS_PATH)\n"

In [8]:
tk = utils.get_tokenizer('spacy')
wv = gensim.models.KeyedVectors.load(EMBED_KEYS_PATH, mmap='r')
null_word = '\0'
d = AbstractDataset('CleanedAVdata.csv', 'Abstract', 'IPCR Classifications', tk, wv.key_to_index,
                    null_word=null_word, min_len=30, verbose=True)

/usr/local/lib/python3.10/dist-packages/torchtext/data/utils.py:105: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(
100%|██████████| 23250/23250 [00:15<00:00, 1497.28it/s]


In [9]:
lr = 1e-3
batch_size = 48
model = CNNBase1(EMBED_KEYS_PATH, null_word=null_word).to(DEVICE)

save_path = f'{HOME}/models/cnn_model-{VERSION}.pth'
act_path = f'{HOME}/models/cnn_model-{VERSION}-max-activations.pkl'
wv_out_path = f'{HOME}/wordvectors/abstracts200_trained_normalized_{VERSION}.wordvectors'

num_pos = d.labels.sum(axis=0, keepdim=True).to_dense()
pos_weight = (d.labels.shape[0] - num_pos) / num_pos
loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight.to(DEVICE))
optimizer = optim.Adam(model.parameters(), lr=lr)

# model, optimizer = load_model_and_opt(model, optimizer, save_path)

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.70)

In [10]:
epochs = 250
train_model(model, optimizer, d, loss_fn, epochs=epochs, batch_size=batch_size, save_freq=25, 
            save_path=save_path, scheduler=scheduler, device=DEVICE)

Learning rate: 0.001
Scheduler: <torch.optim.lr_scheduler.StepLR object at 0x15542eaef580>
Training for 250 epochs, with batch size=48
Using device: cuda:0
Saving model every 25 epochs to /home/hice1/khom9/CSE 8803 BMI Final Project/models/cnn_model-1.pth

-----Epoch 1/250-----
Batch 150/485, loss: 1.407383065422376 (0.896s)
Batch 300/485, loss: 1.3677325503031412 (0.497s)
Batch 450/485, loss: 1.5280050901571909 (0.498s)
Batch 485/485, loss: 1.2095337101391384 (0.124s)
F1 score: 0.3848852972334508

-----Epoch 2/250-----
Batch 150/485, loss: 1.4273061811923982 (0.497s)
Batch 300/485, loss: 1.2728089169661203 (0.494s)
Batch 450/485, loss: 1.3472783998648326 (0.494s)
Batch 485/485, loss: 1.3144826429230827 (0.116s)
F1 score: 0.4018547505709647

-----Epoch 3/250-----
Batch 150/485, loss: 1.1586296979586284 (0.495s)
Batch 300/485, loss: 1.0808591693639755 (0.492s)
Batch 450/485, loss: 1.3127974275747936 (0.492s)
Batch 485/485, loss: 1.0964189972196305 (0.115s)
F1 score: 0.4317589272078272



F1 score: 0.5795351748219533

-----Epoch 33/250-----
Batch 150/485, loss: 0.4150058332830667 (0.494s)
Batch 300/485, loss: 0.33097900440295536 (0.491s)
Batch 450/485, loss: 0.32575667060911656 (0.490s)
Batch 485/485, loss: 0.38343204600470404 (0.114s)
F1 score: 0.5897770519181216

-----Epoch 34/250-----
Batch 150/485, loss: 0.3095662558078766 (0.493s)
Batch 300/485, loss: 0.35924288637936114 (0.491s)
Batch 450/485, loss: 0.3295237411806981 (0.490s)
Batch 485/485, loss: 0.3703146516212395 (0.114s)
F1 score: 0.611008721540474

-----Epoch 35/250-----
Batch 150/485, loss: 0.31012739256024363 (0.492s)
Batch 300/485, loss: 0.30384995403389137 (0.490s)
Batch 450/485, loss: 0.31168015653888387 (0.490s)
Batch 485/485, loss: 0.5687453356172357 (0.115s)
F1 score: 0.6245788982668756

-----Epoch 36/250-----
Batch 150/485, loss: 0.36755525164306163 (0.493s)
Batch 300/485, loss: 0.23648883566260337 (0.490s)
Batch 450/485, loss: 0.35659697058300177 (0.491s)
Batch 485/485, loss: 0.38842276600854736 (0.

Batch 450/485, loss: 0.3073887616644303 (0.491s)
Batch 485/485, loss: 0.22723393477499484 (0.115s)
F1 score: 0.7683184917451736

-----Epoch 66/250-----
Batch 150/485, loss: 0.3831789926439524 (0.494s)
Batch 300/485, loss: 0.18703221843888362 (0.492s)
Batch 450/485, loss: 0.2835490345209837 (0.492s)
Batch 485/485, loss: 0.18947035553199904 (0.115s)
F1 score: 0.7743136516336099

-----Epoch 67/250-----
Batch 150/485, loss: 0.2747685305898388 (0.494s)
Batch 300/485, loss: 0.22395578128596147 (0.491s)
Batch 450/485, loss: 0.3393317119901379 (0.491s)
Batch 485/485, loss: 0.29482312122625964 (0.115s)
F1 score: 0.7673962488685404

-----Epoch 68/250-----
Batch 150/485, loss: 0.3358397975564003 (0.494s)
Batch 300/485, loss: 0.2390079396838943 (0.492s)
Batch 450/485, loss: 0.2671579742555817 (0.492s)
Batch 485/485, loss: 0.21001040350113595 (0.116s)
F1 score: 0.7827009139104966

-----Epoch 69/250-----
Batch 150/485, loss: 0.23379149213433265 (0.493s)
Batch 300/485, loss: 0.27271334127833446 (0.49

F1 score: 0.8574429296163414

-----Epoch 98/250-----
Batch 150/485, loss: 0.23259851648161808 (0.491s)
Batch 300/485, loss: 0.34695525697122015 (0.490s)
Batch 450/485, loss: 0.2263740887803336 (0.489s)
Batch 485/485, loss: 0.23940836143280778 (0.114s)
F1 score: 0.8415471260239781

-----Epoch 99/250-----
Batch 150/485, loss: 0.20237840236475071 (0.491s)
Batch 300/485, loss: 0.2687944476430615 (0.490s)
Batch 450/485, loss: 0.3495612770753602 (0.489s)
Batch 485/485, loss: 0.15440324920096568 (0.114s)
F1 score: 0.8478883266786374

-----Epoch 100/250-----
Batch 150/485, loss: 0.23011758509402475 (0.491s)
Batch 300/485, loss: 0.2906177298973004 (0.491s)
Batch 450/485, loss: 0.27181106564278407 (0.494s)
Batch 485/485, loss: 0.24668631896908794 (0.115s)
Saved to /home/hice1/khom9/CSE 8803 BMI Final Project/models/cnn_model-1.pth
F1 score: 0.8500103873914608

-----Epoch 101/250-----
Batch 150/485, loss: 0.24145625491937 (0.496s)
Batch 300/485, loss: 0.2091798944460849 (0.495s)
Batch 450/485, lo

F1 score: 0.8986864804827123

-----Epoch 130/250-----
Batch 150/485, loss: 0.3164939221056799 (0.492s)
Batch 300/485, loss: 0.15268604772165417 (0.489s)
Batch 450/485, loss: 0.2959321764484048 (0.489s)
Batch 485/485, loss: 0.23813728777957813 (0.115s)
F1 score: 0.9000751906534292

-----Epoch 131/250-----
Batch 150/485, loss: 0.2610199878457934 (0.491s)
Batch 300/485, loss: 0.21191912331618368 (0.490s)
Batch 450/485, loss: 0.2846264705869059 (0.489s)
Batch 485/485, loss: 0.26424434315413237 (0.114s)
F1 score: 0.9033785431543107

-----Epoch 132/250-----
Batch 150/485, loss: 0.2825977773157259 (0.493s)
Batch 300/485, loss: 0.25601031004761654 (0.490s)
Batch 450/485, loss: 0.2608653980990251 (0.489s)
Batch 485/485, loss: 0.08890548686363868 (0.114s)
F1 score: 0.9030453774496167

-----Epoch 133/250-----
Batch 150/485, loss: 0.25956623373242715 (0.492s)
Batch 300/485, loss: 0.2969839307169119 (0.490s)
Batch 450/485, loss: 0.22174161847059926 (0.489s)
Batch 485/485, loss: 0.1897257073117154 (

Batch 150/485, loss: 0.2378459068077306 (0.493s)
Batch 300/485, loss: 0.19146127164984741 (0.490s)
Batch 450/485, loss: 0.3230713261446605 (0.489s)
Batch 485/485, loss: 0.23426851090043782 (0.114s)
F1 score: 0.9227710951689116

-----Epoch 163/250-----
Batch 150/485, loss: 0.2829858940374106 (0.492s)
Batch 300/485, loss: 0.2251232286884139 (0.490s)
Batch 450/485, loss: 0.27144279749753575 (0.490s)
Batch 485/485, loss: 0.11796788752877287 (0.114s)
F1 score: 0.9232140166775802

-----Epoch 164/250-----
Batch 150/485, loss: 0.28981190879518787 (0.491s)
Batch 300/485, loss: 0.2878675990489622 (0.492s)
Batch 450/485, loss: 0.2023795798017333 (0.495s)
Batch 485/485, loss: 0.11358126497694425 (0.115s)
F1 score: 0.924096513263601

-----Epoch 165/250-----
Batch 150/485, loss: 0.2823651225414748 (0.497s)
Batch 300/485, loss: 0.26680626354180276 (0.495s)
Batch 450/485, loss: 0.22002405177491408 (0.494s)
Batch 485/485, loss: 0.15930965856782028 (0.116s)
F1 score: 0.9246278751224847

-----Epoch 166/2

Batch 300/485, loss: 0.2810885322000831 (0.493s)
Batch 450/485, loss: 0.296603719368577 (0.492s)
Batch 485/485, loss: 0.06449656845735652 (0.115s)
F1 score: 0.9317879505539085

-----Epoch 195/250-----
Batch 150/485, loss: 0.2432819292259713 (0.494s)
Batch 300/485, loss: 0.22111562225812426 (0.493s)
Batch 450/485, loss: 0.2589030019038667 (0.492s)
Batch 485/485, loss: 0.3096841770756458 (0.115s)
F1 score: 0.9317001973957636

-----Epoch 196/250-----
Batch 150/485, loss: 0.313543106984968 (0.495s)
Batch 300/485, loss: 0.2375033964837591 (0.493s)
Batch 450/485, loss: 0.1783804824234297 (0.492s)
Batch 485/485, loss: 0.2783825267638479 (0.115s)
F1 score: 0.932469230766485

-----Epoch 197/250-----
Batch 150/485, loss: 0.21846636440294484 (0.495s)
Batch 300/485, loss: 0.2845190263275678 (0.493s)
Batch 450/485, loss: 0.24095750481200714 (0.493s)
Batch 485/485, loss: 0.22076048755220004 (0.115s)
F1 score: 0.9311444221791572

-----Epoch 198/250-----
Batch 150/485, loss: 0.16891736728139223 (0.495

Batch 300/485, loss: 0.22822639731379846 (0.493s)
Batch 450/485, loss: 0.20538683472666888 (0.492s)
Batch 485/485, loss: 0.2746062432016645 (0.115s)
F1 score: 0.9390294714337001

-----Epoch 227/250-----
Batch 150/485, loss: 0.22855848529686532 (0.494s)
Batch 300/485, loss: 0.16557145308858404 (0.492s)
Batch 450/485, loss: 0.34507560472625 (0.491s)
Batch 485/485, loss: 0.20794247355578202 (0.115s)
F1 score: 0.9383035976952715

-----Epoch 228/250-----
Batch 150/485, loss: 0.3001110146365439 (0.494s)
Batch 300/485, loss: 0.266548830276976 (0.491s)
Batch 450/485, loss: 0.16315671956942726 (0.491s)
Batch 485/485, loss: 0.2422189111023077 (0.115s)
F1 score: 0.9377362958450286

-----Epoch 229/250-----
Batch 150/485, loss: 0.2596049742043639 (0.494s)
Batch 300/485, loss: 0.20007752488212038 (0.492s)
Batch 450/485, loss: 0.21412854157698652 (0.491s)
Batch 485/485, loss: 0.4725816511549056 (0.115s)
F1 score: 0.9395430314529355

-----Epoch 230/250-----
Batch 150/485, loss: 0.3787793209264055 (0.4

In [11]:
save_model(save_path, model, optimizer, epochs)
print(f'Saved to {save_path}')

Saved to /home/hice1/khom9/CSE 8803 BMI Final Project/models/cnn_model-1.pth


In [12]:
i = 8721 #18622
txt, label = d[i]
label = label.unsqueeze(0)
print(loss_fn(model(txt.to(DEVICE)).detach(), label.to(DEVICE)).item())
print(torch.cat([model(txt.to(DEVICE)).detach(), label.to(DEVICE)]).T)


5.323808181856293e-06
tensor([[ -28.9006,    0.0000],
        [-314.0933,    0.0000],
        [-252.4385,    0.0000],
        [-258.5004,    0.0000],
        [-152.8970,    0.0000],
        [-298.6354,    0.0000],
        [-238.3664,    0.0000],
        [ -37.4053,    0.0000],
        [ -57.9946,    0.0000],
        [-275.8915,    0.0000],
        [-116.5328,    0.0000],
        [-206.7182,    0.0000],
        [-305.9601,    0.0000],
        [-253.1639,    0.0000],
        [-209.7269,    0.0000],
        [-311.6870,    0.0000],
        [-189.1265,    0.0000],
        [-265.6090,    0.0000],
        [-291.2330,    0.0000],
        [-302.5996,    0.0000],
        [ -21.7756,    0.0000],
        [-312.1428,    0.0000],
        [-232.0197,    0.0000],
        [-298.3479,    0.0000],
        [-274.4721,    0.0000],
        [-312.5026,    0.0000],
        [-216.7874,    0.0000],
        [-212.0948,    0.0000],
        [-303.3674,    0.0000],
        [-283.8804,    0.0000],
        [ -17.4958

In [13]:
pred = (torch.sigmoid(batch_predict(model, d.abst_data, device=DEVICE).detach().cpu()) > 0.5).type(torch.float)
true = d.labels.to_dense()

In [14]:
total_loss = []
loss_fn_cpu = loss_fn.cpu()
for i in range(len(d)):
    total_loss.append(loss_fn_cpu(pred[i].unsqueeze(0), true[i].unsqueeze(0)).item())
    
print(f'Total avg loss: {np.mean(total_loss)}')

Total avg loss: 1.1084296340429656


In [15]:
x = pd.Series(total_loss)
x.sort_values(ascending=False).head(15)

2267     193.282471
22573    186.985901
18414    186.191101
17995    185.235703
18534    181.800720
18548    178.970963
21263    178.187866
17761    177.895233
18622    177.773224
11919     98.311630
20998     97.643471
11239     97.067375
18514     90.423332
18483     90.423332
9062      90.150620
dtype: float64

In [16]:
print(precision_score(true, pred, average=None))
print(f'Total precision: {precision_score(true, pred, average="weighted")}')

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


[0.88817891 0.         0.75       1.         1.         1.
 0.88888889 0.94444444 0.91957105 1.         0.95833333 1.
 1.         0.9        0.8        0.         0.92105263 0.75
 1.         0.94444444 0.91646778 0.         0.5        0.
 0.875      0.         1.         1.         0.         0.5
 0.96189367 0.88679245 0.82608696 0.84210526 0.91082803 0.8961039
 0.91509434 1.         0.         1.         0.3        1.
 0.09090909 0.         0.         0.8        0.         0.
 1.         0.         1.         0.         1.         0.86956522
 0.87179487 1.         0.80357143 0.88095238 0.75       0.77272727
 1.         0.93650794 1.         0.92857143 1.         0.84210526
 1.         0.98113208 0.85       0.77777778 0.         0.88888889
 0.75       0.         0.89034425 0.86062718 0.86538462 0.94736842
 0.93817557 0.93912753 0.88297173 0.8590898  0.82847896 0.8960396
 0.86842105 0.79761905 0.92380952 0.90721649 0.82608696 0.91335265
 0.90816327]
Total precision: 0.9225320781507335


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [17]:
print(recall_score(true, pred, average=None))
print(f'Total recall: {recall_score(true, pred, average="weighted")}')

[0.97887324 0.         0.75       1.         1.         0.5
 0.8        0.95280236 0.95543175 0.92307692 0.96842105 0.52380952
 0.66666667 0.6        0.8        0.         0.67307692 0.14285714
 1.         0.425      0.93203883 0.         0.16666667 0.
 0.29166667 0.         0.36363636 0.76923077 0.         1.
 0.95372051 0.95918367 0.99580713 0.96969697 0.91961415 0.94520548
 0.98979592 1.         0.         0.6        0.5        0.5
 0.2        0.         0.         0.44444444 0.         0.
 1.         0.         0.5        0.         0.84615385 0.90909091
 0.87179487 0.66666667 0.91836735 0.96103896 0.5        0.85
 0.91176471 0.9516129  0.75       0.76470588 0.57142857 0.9039548
 0.66666667 0.94545455 0.77272727 0.73684211 0.         0.8
 0.54545455 0.         0.9906577  0.97244094 0.91836735 0.9
 0.97022566 0.98701754 0.99925595 0.99915086 0.99610895 1.
 1.         1.         0.97487437 0.94964029 0.98701299 0.99569337
 0.93684211]
Total recall: 0.9716742135604716


In [18]:
print(f1_score(true, pred, average='weighted'))

0.945287335378083


In [19]:
print(f'Total accuracy: {accuracy_score(true, pred)}')

Total accuracy: 0.8397849462365592


In [20]:
wv_tuned = gensim.models.KeyedVectors.load(EMBED_KEYS_PATH, mmap='r')
wv_tuned.vectors = model.embedding.weight.data.detach().cpu().numpy()

wv_tuned.vectors = np.clip(wv_tuned.vectors, a_min=0, a_max=1.)

wv_tuned.save(wv_out_path)
print(f'Saved word embeddings to {wv_out_path}')

Saved word embeddings to /home/hice1/khom9/CSE 8803 BMI Final Project/wordvectors/abstracts200_trained_normalized_1.wordvectors


In [21]:
activations = []
model.eval()
with torch.no_grad():
    for txt, label in (d):
        outputs = list(model(txt.to(DEVICE), all_outputs_max=True))
        activations.append(outputs)
activations = torch.tensor(activations)

max_act = torch.max(activations, axis=0)[0]
max_act_dict = OrderedDict(zip(list(dict(model.named_modules()).keys())[1:], max_act))

output = open(act_path, 'wb')
pickle.dump(max_act_dict, output)
output.close()
print(f'Wrote max layer activations to {act_path}')

Wrote max layer activations to /home/hice1/khom9/CSE 8803 BMI Final Project/models/cnn_model-1-max-activations.pkl
